In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/zomato-bangalore-restaurants/zomato.csv")
df.head()   #printing first five rows

In [ ]:
df.shape   #number of rows and columns in dataset

In [ ]:
df.columns   #cursory glance of each column present in dataset 

In [ ]:
len(df.columns)    #total number of columns present in given dataset

In [ ]:
df.dtypes  #Kind of data each column consists

In [ ]:
df.isnull().values.any()    #Does dataset contain any null value if true then will count total null value present in each column

In [ ]:
df.isnull().sum()

#### Now i have got a good understanding of dataset now my next step would be datapreprocessing including modification of data in such a way that it would be understandable by machine

In [ ]:
print("unique value for online_order","\n",df.online_order.unique(),"\n","*"*100,"\n")   #printing list of unique variable present in given feature
print("unique value for book_table","\n",df.book_table.unique())

In [ ]:
#Now i will map yes as 1 and no as 0 and convert it from object type to int type
df.online_order=df.online_order.map({"Yes":1,"No":0}).astype(int)
df.book_table=df.book_table.map({"Yes":1,"No":0}).astype(int)
print(df["online_order"].unique(),"\n","*"*100,"\n",df["book_table"].unique())


In [ ]:
df.dtypes

In [ ]:
#dropping unwanted column
df=df.drop(["url","phone","dish_liked"],axis=1)

In [ ]:
df.head()

In [ ]:
df=df.rename(columns={"approx_cost(for two people)":"cost","listed_in(type)":"type","listed_in(city)":"city"})
df.head()

In [ ]:
df["cost"]=df.cost.str.replace(",","")
df["cost"]

In [ ]:
df=df.dropna(how="any",axis=0)   #dropping all those columns having a any null value

In [ ]:
df.columns

In [ ]:
df.rate.unique()

In [ ]:
df=df.loc[df.rate!="NEW"].reset_index(drop=True)
df=df.loc[df.rate!="-"].reset_index(drop=True)#Removing all the rows from dataframe consists df.rate="-"
df.shape

In [ ]:
df.rate=df.rate.str.replace("/5","")
df.rate=df.rate.str.strip().astype(float)
df.rate.unique()


In [ ]:
def Encode(df):
    for i in df.columns[~df.columns.isin(['rate', 'cost', 'votes','book_table','online_order'])]:
        df[i] = df[i].factorize()[0]
    return df

df_en = Encode(df.copy())
df_en.head() 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,8))
corr=df_en.corr()
sns.heatmap(corr,linewidth=0.3,annot=True,cmap="RdGy_r")

### Here highest number of correlation is in between name  and address hence they will not have much concern in modeling

In [ ]:
df.head()

### Now i will do data visualization operation on modified dataset

In [ ]:
#Whether Restraunt delivering online food or not
plt.figure(figsize=(10,8))
sns.countplot(df.online_order)

more than 25000 delivery has been done online and online food delivery has humoungous impact over customer as compared to offline delivery. People are more comfortable with online service.

In [ ]:
#Does restraunt allow advance table booking or not
plt.figure(figsize=(10,8))
sns.countplot(y=df.book_table)

A lot of restraunt which has not adopted table booking service via zomato.

In [ ]:
#cross tab is used to make relation between two variable here i will try to correlate relation between table booking rate  vs normal rate
plt.figure(figsize=(25,25))
y=pd.crosstab(df.rate,df.book_table)
y.plot(kind="bar",stacked=True)


In [ ]:
#checking type of service
sns.countplot(df.type).set_xticklabels(sns.countplot(df.type).get_xticklabels(),rotation=90,ha="right")

In [ ]:
#location of service
plt.figure(figsize=(20,20))
sns.countplot(df.location).set_xticklabels(sns.countplot(df.location).get_xticklabels(),rotation=90,ha="right")
plt.tight_layout()

In [ ]:
#cost of restraunt base on type of service
sns.countplot(df.cost).set_xticklabels(sns.countplot(df.type).get_xticklabels(),rotation=45,ha="right")
plt.tight_layout()

Highest amount of contribution has been seen by delivery service and for pubs and bars service is less demanded

In [ ]:
#Counting number of items sold based on cost
plt.figure(figsize=(12,8))
sns.countplot(df.cost).set_xticklabels(sns.countplot(df.cost).get_xticklabels(),rotation=90,ha="right")
plt.tight_layout()

In [ ]:
#Rating based on city
plt.figure(figsize=(12,8))
Rating=pd.crosstab(df.rate,df.city)
Rating
Rating.plot(kind="bar",stacked=True)

Max number of item has been sold with cost ranging between 300-500

In [ ]:
#counting online order based according to city
plt.figure(figsize=(12,8))
sns.countplot(df.city).set_xticklabels(sns.countplot(df.city).get_xticklabels(),rotation=90,ha="right")
plt.show()

Customers belonging  to BTM city have been ordered maximum number of time

In [ ]:
#Top 10  most famous restraunt chain
plt.figure(figsize=(12,8))
rest=df.name.value_counts().head(10)
sns.barplot(rest,rest.index)

Here caffe coffe day has emerged as most demanded restraunt service